In [1]:
!pip install -q google-adk


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

In [3]:
from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import InMemoryRunner
from google.genai import types

# Multi-Agent Systems with Google ADK

So far we've built single agents — one LLM with tools. But complex tasks often need **multiple agents** working together, each with their own specialty.

Google ADK supports this with **sub-agents**. Two key patterns:

| Pattern | How it works | When to use |
|---|---|---|
| **SequentialAgent** | Runs sub-agents in order, like a pipeline | Fixed workflows (fetch → analyze → report) |
| **LLM Coordinator** | A parent agent delegates to sub-agents based on the user's request | Routing / helpdesk / triage scenarios |

We'll build both in this notebook.

---

## Part 1: Analytics Report Generator (Sequential Pipeline)

We'll build a 3-stage pipeline:

```
DataFetcher → Analyzer → ReportWriter
```

Each agent writes its output to shared state via `output_key`, and the next agent picks it up.

### Fake sales data

In [4]:
SALES_DATA = {
    "2024-Q3": [
        {"product": "Widget A", "units": 1200, "revenue": 36000},
        {"product": "Widget B", "units": 800, "revenue": 48000},
        {"product": "Gadget X", "units": 450, "revenue": 67500},
        {"product": "Gadget Y", "units": 300, "revenue": 27000},
    ],
    "2024-Q4": [
        {"product": "Widget A", "units": 1400, "revenue": 42000},
        {"product": "Widget B", "units": 950, "revenue": 57000},
        {"product": "Gadget X", "units": 520, "revenue": 78000},
        {"product": "Gadget Y", "units": 280, "revenue": 25200},
    ],
}

### Tools for each agent

In [5]:
# --- DataFetcher tools ---

def get_sales_data(quarter: str) -> str:
    """Fetch sales data for a given quarter (e.g. '2024-Q3'). Returns a list of product sales records."""
    data = SALES_DATA.get(quarter)
    if data:
        return str(data)
    return f"No data found for {quarter}. Available quarters: {list(SALES_DATA.keys())}"


def list_available_quarters() -> str:
    """List all quarters that have sales data available."""
    return str(list(SALES_DATA.keys()))


# --- Analyzer tools ---

def compute_growth(current_quarter: str, previous_quarter: str) -> str:
    """Compute revenue growth rate between two quarters. Returns per-product and total growth."""
    curr = SALES_DATA.get(current_quarter)
    prev = SALES_DATA.get(previous_quarter)
    if not curr or not prev:
        return "One or both quarters not found."
    prev_map = {r["product"]: r["revenue"] for r in prev}
    results = []
    for r in curr:
        old = prev_map.get(r["product"], 0)
        growth = ((r["revenue"] - old) / old * 100) if old else 0
        results.append(f"{r['product']}: ${old:,} → ${r['revenue']:,} ({growth:+.1f}%)")
    total_prev = sum(r["revenue"] for r in prev)
    total_curr = sum(r["revenue"] for r in curr)
    total_growth = (total_curr - total_prev) / total_prev * 100
    results.append(f"TOTAL: ${total_prev:,} → ${total_curr:,} ({total_growth:+.1f}%)")
    return "\n".join(results)


def find_top_products(quarter: str, metric: str) -> str:
    """Find top products for a quarter ranked by a metric ('revenue' or 'units'). Returns ranked list."""
    data = SALES_DATA.get(quarter)
    if not data:
        return f"No data for {quarter}"
    if metric not in ("revenue", "units"):
        return "Metric must be 'revenue' or 'units'"
    ranked = sorted(data, key=lambda r: r[metric], reverse=True)
    return "\n".join(f"{i+1}. {r['product']}: {r[metric]:,} {metric}" for i, r in enumerate(ranked))


# --- ReportWriter has no tools (just writes from context) ---

### Define the 3 sub-agents

In [6]:
data_fetcher = Agent(
    model="gemini-3-flash-preview",
    name="data_fetcher",
    instruction="""You are a data fetcher. Your job is to retrieve sales data.

1. First list available quarters.
2. Then fetch sales data for ALL available quarters.
3. Compile everything you retrieved into your response.""",
    tools=[get_sales_data, list_available_quarters],
    output_key="fetched_data",
)

analyzer = Agent(
    model="gemini-3-flash-preview",
    name="analyzer",
    instruction="""You are a data analyst. The fetched sales data is here:
{fetched_data}

Analyze it:
1. Compute growth between the two quarters using the compute_growth tool.
2. Find top products by revenue for the latest quarter.
3. Summarize your findings.""",
    tools=[compute_growth, find_top_products],
    output_key="analysis",
)

report_writer = Agent(
    model="gemini-3-flash-preview",
    name="report_writer",
    instruction="""You are a report writer. Using the analysis below, write a concise executive summary.

Analysis:
{analysis}

Format the report with:
- A headline with the quarter
- Key metrics (total revenue, growth rate)
- Top performing products
- One actionable recommendation

Keep it under 200 words.""",
    output_key="report",
)

### Wire into a SequentialAgent pipeline

In [7]:
pipeline = SequentialAgent(
    name="analytics_pipeline",
    sub_agents=[data_fetcher, analyzer, report_writer],
)

### Run the pipeline

In [8]:
runner = InMemoryRunner(agent=pipeline, app_name="analytics_pipeline")
runner.auto_create_session = True
user_content = types.Content(
    role="user",
    parts=[types.Part.from_text(text="Generate a quarterly analytics report.")],
)

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                final_text = part.text

print(final_text)

_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 49.180331848s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_r

### What just happened?

```
User: "Generate a quarterly analytics report."
  │
  ▼
┌─────────────────────────────────────────────┐
│           SequentialAgent (pipeline)         │
│                                              │
│  ┌──────────────┐   output_key:              │
│  │ DataFetcher   │──→ fetched_data           │
│  └──────────────┘         │                  │
│                           ▼                  │
│  ┌──────────────┐   output_key:              │
│  │ Analyzer      │──→ analysis               │
│  └──────────────┘         │                  │
│                           ▼                  │
│  ┌──────────────┐   output_key:              │
│  │ ReportWriter  │──→ report                 │
│  └──────────────┘                            │
└─────────────────────────────────────────────┘
```

Key ideas:
- **`output_key`** saves each agent's final response into shared state (like a dict)
- **`{fetched_data}`** in the next agent's instruction reads from that state
- The `SequentialAgent` runs them in order — no LLM decides the routing

---

## Part 2: Exercise — Build a Multi-Agent Analytics Helpdesk

Now you'll build a **coordinator + specialist** pattern. Instead of a fixed pipeline, an LLM coordinator routes user queries to the right specialist sub-agent.

```
User question
  │
  ▼
┌─────────────────────┐
│  Coordinator Agent   │  ← LLM decides who to call
│  (sub_agents=[...])  │
└──┬──────────┬────────┘
   │          │
   ▼          ▼
┌──────┐  ┌──────────────┐
│ SQL  │  │ Data Quality │
│ Agent│  │ Agent        │
└──────┘  └──────────────┘
```

The coordinator is just an `Agent` with `sub_agents=[...]`. The model's instruction tells it when to delegate to which specialist.

### Fake database for the exercise

In [ ]:
TABLES = {
    "customers": [
        {"id": 1, "name": "Acme Corp", "segment": "Enterprise", "mrr": 12000},
        {"id": 2, "name": "StartupXYZ", "segment": "SMB", "mrr": 500},
        {"id": 3, "name": "MegaRetail", "segment": "Enterprise", "mrr": 25000},
        {"id": 4, "name": "LocalShop", "segment": "SMB", "mrr": 200},
        {"id": 5, "name": "TechGiant", "segment": "Enterprise", "mrr": 45000},
    ],
    "events": [
        {"customer_id": 1, "event": "login", "count": 340, "month": "2024-12"},
        {"customer_id": 1, "event": "export", "count": 45, "month": "2024-12"},
        {"customer_id": 2, "event": "login", "count": 12, "month": "2024-12"},
        {"customer_id": 3, "event": "login", "count": 580, "month": "2024-12"},
        {"customer_id": 3, "event": "export", "count": 120, "month": "2024-12"},
        {"customer_id": 4, "event": "login", "count": 3, "month": "2024-12"},
        {"customer_id": 5, "event": "login", "count": 890, "month": "2024-12"},
        {"customer_id": 5, "event": "export", "count": 200, "month": "2024-12"},
    ],
}

### TODO 1: Define tools for the SQL Agent

The SQL agent helps analysts query data. Create two tools:
- `list_tables()` — returns available table names
- `run_query(table: str, filter_field: str, filter_value: str)` — filters a table and returns matching rows

Hint: Use the `TABLES` dict above. The "query" is just a Python dict filter — no real SQL needed.

In [ ]:
# TODO 1: Define tools for the SQL Agent

def list_tables() -> str:
    """List all available database tables."""
    # TODO: Return the table names from TABLES
    pass


def run_query(table: str, filter_field: str, filter_value: str) -> str:
    """Query a table with an optional filter. Returns matching rows.

    Args:
        table: Name of the table to query (e.g. 'customers')
        filter_field: Column to filter on (e.g. 'segment')
        filter_value: Value to match (e.g. 'Enterprise')
    """
    # TODO: Look up the table in TABLES, filter rows where row[filter_field] matches filter_value,
    #       and return the matching rows as a string. Handle missing tables gracefully.
    pass

### TODO 2: Define tools for the Data Quality Agent

The data quality agent checks datasets for issues. Create two tools:
- `check_missing_values(table: str)` — checks for None/empty values in any field
- `check_duplicates(table: str, field: str)` — checks if any values in a field are repeated

In [ ]:
# TODO 2: Define tools for the Data Quality Agent

def check_missing_values(table: str) -> str:
    """Check a table for missing or None values in any field. Returns a report of findings."""
    # TODO: Iterate through rows in TABLES[table], check each field for None or empty string,
    #       and return a summary (e.g. "No missing values" or "Row 2: field 'name' is missing")
    pass


def check_duplicates(table: str, field: str) -> str:
    """Check if any values in a specific field are duplicated.

    Args:
        table: Name of the table to check
        field: Column name to check for duplicates
    """
    # TODO: Collect all values for the given field, find duplicates, and return a report
    pass

### TODO 3: Create the specialist sub-agents

Create two `Agent` instances:
- `sql_agent` — with the SQL tools and an instruction saying it helps run queries
- `data_quality_agent` — with the quality tools and an instruction about checking data health

In [ ]:
# TODO 3: Create specialist sub-agents

sql_agent = Agent(
    model="gemini-3-flash-preview",
    name="sql_agent",
    # TODO: Write an instruction telling this agent it's a SQL specialist that helps
    #       analysts query data. It should list tables first, then run queries.
    instruction="""
    """,
    tools=[],  # TODO: Add your SQL tools here
)

data_quality_agent = Agent(
    model="gemini-3-flash-preview",
    name="data_quality_agent",
    # TODO: Write an instruction telling this agent it's a data quality specialist
    #       that checks for missing values and duplicates.
    instruction="""
    """,
    tools=[],  # TODO: Add your data quality tools here
)

### TODO 4: Create the coordinator agent

The coordinator doesn't have tools — it has `sub_agents`. Its instruction tells the LLM when to route to each specialist.

Key: Use `sub_agents=[...]` instead of `tools=[...]`.

In [ ]:
# TODO 4: Create the coordinator agent

coordinator = Agent(
    model="gemini-3-flash-preview",
    name="analytics_helpdesk",
    # TODO: Write an instruction that tells the coordinator to:
    #       - Route data/query questions to sql_agent
    #       - Route quality/validation questions to data_quality_agent
    #       - Answer general analytics questions itself
    instruction="""
    """,
    sub_agents=[],  # TODO: Add your specialist agents here
)

### TODO 5: Test with a user query

Try different queries to see the coordinator route to different specialists:
- `"Show me all enterprise customers"` → should go to SQL agent
- `"Are there any data quality issues in the events table?"` → should go to data quality agent
- `"What metrics should I track for churn?"` → coordinator answers directly

In [ ]:
# TODO 5: Change this query to test different routing paths

runner = InMemoryRunner(agent=coordinator, app_name="analytics_helpdesk")
runner.auto_create_session = True

test_query = "Show me all enterprise customers"  # TODO: Try different queries!

user_content = types.Content(
    role="user",
    parts=[types.Part.from_text(text=test_query)],
)

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                final_text = part.text

print(final_text)

## Recap

| Pattern | ADK Feature | You Built |
|---|---|---|
| **Sequential pipeline** | `SequentialAgent` + `output_key` | Analytics report generator |
| **LLM-routed delegation** | `Agent` with `sub_agents` | Analytics helpdesk |

Both patterns use the same building block — `Agent` with tools — but compose them differently. The ADK handles all the message passing and routing mechanics.